# Code-to-Code Comparison: Dinwoodie

### National Renewable Energy Laboratory

#### Rob Hammond

##### 27 May 2021

In [1]:
import os
from copy import deepcopy
from time import perf_counter
from pprint import pprint

import yaml
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from wombat.core import Simulation
from wombat.core.library import DINWOODIE, load_yaml

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

In [2]:
# Converting Labor values to fixed cost input for the base case
tech_salary_annual = 80000
techs = 20
capacity = 240 * 1000  # 240 -> kW
f"{tech_salary_annual * techs / capacity:.4f}"

'6.6667'

In [3]:
configs = [
    "base",
    "more_ctvs",
    "fewer_ctvs",
    "more_techs",
    "fewer_techs",
    "failure_50",
    "failure_200",
    "no_hlvs",
    "no_weather",
    "historic_weather",
    "manual_resets_only",
    "minor_repairs_only",
    "medium_repairs_only",
    "major_repairs_only",
    "major_replacements_only",
    "annual_service_only",
]
columns = deepcopy(configs)
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "power production": [],
    "task completion rate": [],
    "annual direct O&M cost": [],
    "annual vessel cost": [],
    "ctv cost": [],
    "fsv cost": [],
    "hlv cost": [],
    "annual repair cost": [],
    "annual technician cost": [],
    "ctv utilization": [],
    "fsv utilization": [],
    "hlv utilization": [],
    
}

In [4]:
for config in configs:
    # Run the simulation
    start = perf_counter()
    sim = Simulation(DINWOODIE, f"{config}.yaml")
    sim.run()
    end = perf_counter()
    print(f"{sim.config.name.rjust(30)} | {(end - start) / 60:.2f} m")
    
    # Gather the results of interest
    years = sim.metrics.events.year.unique().shape[0]
    mil = 1000000
    
    availability_time = sim.metrics.time_based_availability(frequency="project", by="windfarm")
    availability_production = sim.metrics.production_based_availability(frequency="project", by="windfarm")
    cf_net = sim.metrics.capacity_factor(which="net", frequency="project", by="windfarm")
    cf_gross = sim.metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
    power_production = sim.metrics.power_production(frequency="project", by_turbine=False).values[0][0]
    completion_rate = sim.metrics.task_completion_rate(which="both", frequency="project")
    parts = sim.metrics.events[["materials_cost"]].sum().sum()
    techs = sim.metrics.project_fixed_costs(frequency="project", resolution="low").operations[0]
    total = sim.metrics.events[["total_cost"]].sum().sum()
    
    equipment = sim.metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    hlv = equipment[[el for el in equipment.columns if "Heavy Lift Vessel" in el]].sum().sum()
    fsv = equipment[[el for el in equipment.columns if "Field Support Vessel" in el]].sum().sum()
    ctv = equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]].sum().sum()
    
    utilization = sim.metrics.service_equipment_utilization(frequency="project")
    hlv_ur = utilization[[el for el in utilization.columns if "Heavy Lift Vessel" in el]].mean().mean()
    fsv_ur = utilization[[el for el in utilization.columns if "Field Support Vessel" in el]].mean().mean()
    ctv_ur = utilization[[el for el in utilization.columns if "Crew Transfer Vessel" in el]].mean().mean()
    
    # Log the results of interest
    results["availability - time based"].append(availability_time)
    results["availability - production based"].append(availability_production)
    results["capacity factor - net"].append(cf_net)
    results["capacity factor - gross"].append(cf_gross)
    results["power production"].append(power_production)
    results["task completion rate"].append(completion_rate)
    results["annual direct O&M cost"].append((total + techs) / mil / years)
    results["annual vessel cost"].append(equipment_sum / mil / years)
    results["ctv cost"].append(ctv / mil / years)
    results["fsv cost"].append(fsv / mil / years)
    results["hlv cost"].append(hlv / mil / years)
    results["annual repair cost"].append(parts / mil / years)
    results["annual technician cost"].append(techs / mil / years)
    results["ctv utilization"].append(ctv_ur)
    results["fsv utilization"].append(fsv_ur)
    results["hlv utilization"].append(hlv_ur)
    sim.env.cleanup_log_files(log_only=True)

                dinwoodie_base | 2.74 m
           dinwoodie_more_ctvs | 2.94 m
          dinwoodie_fewer_ctvs | 2.95 m
          dinwoodie_more_techs | 2.86 m
         dinwoodie_fewer_techs | 2.85 m
          dinwoodie_failure_50 | 2.46 m
         dinwoodie_failure_200 | 3.79 m
             dinwoodie_no_hlvs | 3.28 m
          dinwoodie_no_weather | 3.05 m
    dinwoodie_historic_weather | 2.57 m
  dinwoodie_manual_resets_only | 2.35 m
  dinwoodie_minor_repairs_only | 2.02 m
 dinwoodie_medium_repairs_only | 1.82 m
  dinwoodie_major_repairs_only | 1.73 m
dinwoodie_major_replacements_only | 1.87 m
 dinwoodie_annual_service_only | 2.40 m


In [5]:
# Save the results
# pickled dictionary format
with open(DINWOODIE / "outputs" / "results_dict_v0.4.yaml", "w") as f:
    yaml.dump(results, f, default_flow_style=False, sort_keys=False)

# dataframe/csv format
results_df = pd.DataFrame(results.values(), columns=columns, index=results.keys()).fillna(0)
results_df.to_csv(DINWOODIE / "outputs" / "results_data_v0.4.csv", index_label="result")

In [6]:
results_df

,base,more_ctvs,fewer_ctvs,more_techs,fewer_techs,failure_50,failure_200,no_hlvs,no_weather,historic_weather,manual_resets_only,minor_repairs_only,medium_repairs_only,major_repairs_only,major_replacements_only,annual_service_only
availability - time based,0.96,0.95,0.94,0.94,0.95,0.97,0.80,0.98,0.97,0.93,1.00,1.00,1.00,1.00,0.98,0.98
availability - production based,0.96,0.95,0.94,0.94,0.95,0.97,0.80,0.98,0.00,0.93,1.00,1.00,1.00,1.00,0.98,0.98
capacity factor - net,0.46,0.45,0.45,0.45,0.45,0.46,0.38,0.47,0.00,0.45,0.48,0.48,0.48,0.48,0.47,0.47
capacity factor - gross,0.48,0.48,0.48,0.48,0.48,0.48,0.48,0.48,0.00,0.48,0.48,0.48,0.48,0.48,0.48,0.48
power production,"9,585,848,704.00","9,501,458,133.00","9,456,354,933.00","9,454,324,661.50","9,529,070,457.00","9,767,366,551.00","8,052,262,327.00","9,828,534,478.00",0.00,"7,522,832,761.50","9,999,703,956.00","9,999,125,331.50","10,015,665,714.50","10,029,768,220.50","9,789,479,833.50","9,870,342,131.00"
task completion rate,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.97,0.96,1.00
annual direct O&M cost,16.97,20.77,17.02,17.83,16.42,14.97,18.10,6.07,16.23,16.89,3.52,3.76,3.92,4.02,14.70,5.07
annual vessel cost,11.84,13.17,11.82,11.78,11.92,10.90,12.19,2.18,10.82,11.73,1.92,1.92,1.92,2.18,11.40,1.92
ctv cost,1.92,3.20,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92,1.92
fsv cost,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.27,0.00,0.00,0.00,0.27,0.00,0.00
